## A Storage-Focused Chatbot
Answer all your SAS, SATA, and NVMe questions


In [4]:
# Load environment variables
import os
from dotenv import load_dotenv,find_dotenv

if find_dotenv():
    print("Found .env file")
    load_dotenv(find_dotenv())  
else:
    print("No .env file found")
# print all environment variables
for key, value in os.environ.items():
    if 'API' in key:
        print(f"{key}: {value}")

No .env file found


In [ ]:
# import schema for chat messages and ChatOpenAI in order to query chatmodels GPT-3.5-turbo or GPT-4

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI


In [ ]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo",temperature=0.3)
messages = [
    SystemMessage(content="You are an expert data scientist"),
    HumanMessage(content="Write a Python script that trains a neural network on simulated data ")
]
response=chat(messages)

print(response.content,end='\n')